In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, History, TensorBoard
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import RMSprop
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model

Using TensorFlow backend.


In [2]:
#parameters
classes = 12
batch_size=32
train_total = 3783
validation_total = 967

In [3]:
model_mobilenet = MobileNet(include_top=False, weights = 'imagenet',input_shape=(224,224,3), alpha=0.5)

In [4]:
model_mobilenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 16)      0         
__________

In [5]:
x = model_mobilenet.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)

In [6]:
model = Model(inputs=model_mobilenet.input, outputs=x)

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 16)      0         
__________

In [8]:
print(model.layers[0].trainable)

False


In [9]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range = 90,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
train_generator = train_datagen.flow_from_directory(
                    'train_second_split',
                    target_size=(224,224),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation_second_split',
                        target_size=(224,224),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 3783 images belonging to 12 classes.
Found 967 images belonging to 12 classes.


In [11]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])

In [16]:
model.load_weights('mobilenet_from_scratch_save_best_final_2_96977.hdf5')

In [18]:
checkpointer = ModelCheckpoint(filepath='mobilenet_from_scratch_second_split.hdf5', verbose=1, save_best_only=True, save_weights_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0, verbose=1)
earlystopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

In [31]:
history = model.fit_generator(
                    train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2,
                    callbacks=[checkpointer, reduceLR, earlystopping])

Epoch 1/100
Epoch 00001: val_loss improved from inf to 1.37241, saving model to mobilenet_from_scratch_second_split_augment.hdf5
 - 83s - loss: 2.1576 - acc: 0.3282 - val_loss: 1.3724 - val_acc: 0.5657
Epoch 2/100
Epoch 00002: val_loss improved from 1.37241 to 0.73050, saving model to mobilenet_from_scratch_second_split_augment.hdf5
 - 73s - loss: 1.1891 - acc: 0.6082 - val_loss: 0.7305 - val_acc: 0.7818
Epoch 3/100
Epoch 00003: val_loss improved from 0.73050 to 0.41916, saving model to mobilenet_from_scratch_second_split_augment.hdf5
 - 73s - loss: 0.7814 - acc: 0.7413 - val_loss: 0.4192 - val_acc: 0.8738
Epoch 4/100
Epoch 00004: val_loss improved from 0.41916 to 0.33870, saving model to mobilenet_from_scratch_second_split_augment.hdf5
 - 71s - loss: 0.6015 - acc: 0.7934 - val_loss: 0.3387 - val_acc: 0.8800
Epoch 5/100
Epoch 00005: val_loss improved from 0.33870 to 0.28229, saving model to mobilenet_from_scratch_second_split_augment.hdf5
 - 73s - loss: 0.4782 - acc: 0.8431 - val_loss:

KeyboardInterrupt: 

In [19]:
model.load_weights('mobilenet_from_scratch_second_split.hdf5')

In [20]:
model.evaluate_generator(validation_generator, int(np.ceil(validation_total/batch_size)))

[0.069503415924671408, 0.97518097207859356]

In [21]:
predictions_valid_values = model.predict_generator(validation_generator,int(np.ceil(validation_total/batch_size)))

In [22]:
predictions_valid = np.argmax(predictions_valid_values, axis=1)


In [23]:
true_positive = np.zeros(12)
false_positive = np.zeros(12)
true_negative = np.zeros(12)
false_negative = np.zeros(12)
for i in range(len(predictions_valid)):
    if(predictions_valid[i]==validation_generator.classes[i]):
        true_positive[predictions_valid[i]] = true_positive[predictions_valid[i]] + 1
    else:
        false_positive[predictions_valid[i]] = false_positive[predictions_valid[i]] + 1
        false_negative[validation_generator.classes[i]] = false_negative[validation_generator.classes[i]] +1

In [24]:
print('true_positive')
print(true_positive)
print('false_positive')
print(false_positive)
print('false_negative')
print(false_negative)
print('Total')
print(true_positive+false_negative)

true_positive
[  41.   92.   58.  132.   46.   77.  127.   44.  107.   44.   93.   82.]
false_positive
[  8.   0.   0.   0.   1.   1.  11.   0.   2.   1.   0.   0.]
false_negative
[ 8.  0.  1.  1.  1.  2.  8.  0.  1.  1.  0.  1.]
Total
[  49.   92.   59.  133.   47.   79.  135.   44.  108.   45.   93.   83.]


In [19]:
model_mobilenet_2 = MobileNet(include_top=False, weights = 'imagenet',input_shape=(224,224,3), alpha=0.5)
x = model_mobilenet_2.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)
model_2 = Model(inputs=model_mobilenet_2.input, outputs=x)

In [20]:
model_2.load_weights('mobilenet_from_scratch_save_best_final_2_96977.hdf5')

In [59]:
#averaging the weights from two splits

In [25]:
for i in range(len(model.layers)):
    res_weights = [(model.layers[i].get_weights()[j]+model_2.layers[i].get_weights()[j])/2 for j in range(len(model.layers[i].get_weights()))]
    model.layers[i].set_weights(res_weights)

In [21]:
model.evaluate_generator(validation_generator, int(np.ceil(validation_total/batch_size)))

[0.069503415924671408, 0.97518097207859356]

In [22]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(224,224),
                    batch_size=batch_size,
                    class_mode=None)


Found 794 images belonging to 1 classes.


In [39]:
predictions = model.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes = [class_ids[x] for x in np.argmax(predictions, axis=1)]


In [40]:
predictions_2 = model_2.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes_2 = [class_ids[x] for x in np.argmax(predictions_2, axis=1)]


In [41]:
predictions_3 = (predictions + predictions_2)/2

In [42]:
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes_3 = [class_ids[x] for x in np.argmax(predictions_3, axis=1)]


In [ ]:
from os.path import basename
test_ids = [basename(x) for x in test_generator.filenames]
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes})
submission.to_csv('submission_mobilenet_from_scratch_second_split.csv', encoding="utf8", index=False)

In [ ]:
from IPython.display import FileLink
FileLink('submission_mobilenet_from_scratch_second_split.csv')

In [31]:
from os.path import basename
test_ids = [basename(x) for x in test_generator.filenames]
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes})
submission.to_csv('submission_mobilenet_from_scratch_second_split_outputs_average.csv', encoding="utf8", index=False)

In [32]:
from IPython.display import FileLink
FileLink('submission_mobilenet_from_scratch_second_split_outputs_average.csv')

/home/janardhan/plant_seeding/submission_mobilenet_from_scratch_second_split_outputs_average.csv

In [26]:
# PB LB: 0.97355 - weights averaged
# PB LB: 0.96851 - output predictions averaged